In [13]:
import os

import pickle
import keras
import numpy as np
import tensorflow as tf
import torch
import xgboost as xgb

from PIL import Image
from scipy import stats
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
from keras.models import model_from_json

from datetime import datetime

In [9]:

def model(X_train, y_train, X_test, y_test) -> xgb.Booster:
	results = {}
	dtrain = xgb.DMatrix(X_train, label=y_train)
	dtest = xgb.DMatrix(X_test, label=y_test)

	params = {
		'max_depth':12,
		'eta':0.05,
		'objective':'multi:softprob',
		'num_class':10,
		'early_stopping_rounds':10,
		'eval_metric':'merror'
	}

	watchlist = [(dtrain, 'train'),(dtest, 'eval')]
	n_round = 400

	model = xgb.train(
		params,
		dtrain,
		n_round,
		watchlist,
		evals_result=results
    )
	pickle.dump(model, open('model.pickle.dat', 'wb'))

	return model

In [38]:
def load_images_from_folder(img_dir: str) -> list[str]:
    images = list()

    for file in os.listdir(img_dir):
        file = os.path.join(img_dir, file)

        if not os.path.isfile(file):  # Ignore directories etc.
            continue

        img = Image.open(file)
        if img is not None:
            img = np.asarray(img)
            # img = torch.from_numpy(img)
            images.append(img)
    
    return images

In [34]:
from keras.datasets import cifar10
(fake_X, fake_y), (_, _) = cifar10.load_data()

fake_X

array([[[[ 59,  62,  63],
         [ 43,  46,  45],
         [ 50,  48,  43],
         ...,
         [158, 132, 108],
         [152, 125, 102],
         [148, 124, 103]],

        [[ 16,  20,  20],
         [  0,   0,   0],
         [ 18,   8,   0],
         ...,
         [123,  88,  55],
         [119,  83,  50],
         [122,  87,  57]],

        [[ 25,  24,  21],
         [ 16,   7,   0],
         [ 49,  27,   8],
         ...,
         [118,  84,  50],
         [120,  84,  50],
         [109,  73,  42]],

        ...,

        [[208, 170,  96],
         [201, 153,  34],
         [198, 161,  26],
         ...,
         [160, 133,  70],
         [ 56,  31,   7],
         [ 53,  34,  20]],

        [[180, 139,  96],
         [173, 123,  42],
         [186, 144,  30],
         ...,
         [184, 148,  94],
         [ 97,  62,  34],
         [ 83,  53,  34]],

        [[177, 144, 116],
         [168, 129,  94],
         [179, 142,  87],
         ...,
         [216, 184, 140],
        

In [27]:
fake_y

array([[6],
       [9],
       [9],
       ...,
       [9],
       [1],
       [1]], dtype=uint8)

In [39]:
IMAGE_PATH = './50-50/'

X = load_images_from_folder(IMAGE_PATH + 'fire')
X.extend(load_images_from_folder(IMAGE_PATH + 'no-fire'))

X

[array([[[75, 16, 10],
         [74, 15,  9],
         [73, 14,  8],
         ...,
         [19,  7,  7],
         [18,  6,  6],
         [18,  6,  6]],
 
        [[75, 16, 10],
         [74, 15,  9],
         [73, 14,  8],
         ...,
         [19,  7,  7],
         [18,  6,  6],
         [18,  6,  6]],
 
        [[75, 16, 10],
         [74, 15,  9],
         [75, 14,  9],
         ...,
         [19,  7,  7],
         [18,  6,  6],
         [18,  6,  6]],
 
        ...,
 
        [[96, 39, 30],
         [88, 34, 22],
         [90, 33, 22],
         ...,
         [34, 10,  6],
         [32, 11, 10],
         [32, 11, 10]],
 
        [[61, 10,  6],
         [62, 13,  6],
         [65, 15,  8],
         ...,
         [38, 14, 10],
         [32, 11, 10],
         [32, 11, 10]],
 
        [[51,  3,  0],
         [53,  5,  1],
         [54,  7,  1],
         ...,
         [36, 12,  8],
         [32, 11, 10],
         [32, 11, 10]]], dtype=uint8),
 array([[[ 43,  43,  43],
         [ 30,  

In [23]:
# 0 == 'fire', 1 == 'no-fire'
y = [[0]] * 500
y.extend([[1]] * 500)

y = np.asarray(y, dtype='uint8')
y

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
    

In [40]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42, test_size=0.4
)

X_train

[array([[[103, 101, 102],
         [103, 101, 102],
         [103, 101, 102],
         ...,
         [125, 125, 125],
         [122, 122, 122],
         [122, 122, 122]],
 
        [[104, 102, 103],
         [104, 102, 103],
         [104, 102, 103],
         ...,
         [125, 125, 125],
         [124, 124, 124],
         [124, 124, 124]],
 
        [[103, 101, 102],
         [103, 101, 102],
         [104, 102, 103],
         ...,
         [125, 125, 125],
         [127, 127, 127],
         [127, 127, 127]],
 
        ...,
 
        [[ 54,  34,  23],
         [ 55,  35,  24],
         [ 55,  35,  24],
         ...,
         [ 25,  19,  21],
         [ 23,  17,  19],
         [ 23,  17,  19]],
 
        [[ 55,  33,  22],
         [ 55,  33,  22],
         [ 56,  34,  23],
         ...,
         [ 24,  18,  18],
         [ 22,  18,  17],
         [ 22,  18,  17]],
 
        [[ 55,  33,  22],
         [ 55,  33,  22],
         [ 56,  34,  23],
         ...,
         [ 24,  19,  16],
  

In [41]:
X_train = np.array(X_train)
X_test = np.array(X_test)
X_train

array([[[[103, 101, 102],
         [103, 101, 102],
         [103, 101, 102],
         ...,
         [125, 125, 125],
         [122, 122, 122],
         [122, 122, 122]],

        [[104, 102, 103],
         [104, 102, 103],
         [104, 102, 103],
         ...,
         [125, 125, 125],
         [124, 124, 124],
         [124, 124, 124]],

        [[103, 101, 102],
         [103, 101, 102],
         [104, 102, 103],
         ...,
         [125, 125, 125],
         [127, 127, 127],
         [127, 127, 127]],

        ...,

        [[ 54,  34,  23],
         [ 55,  35,  24],
         [ 55,  35,  24],
         ...,
         [ 25,  19,  21],
         [ 23,  17,  19],
         [ 23,  17,  19]],

        [[ 55,  33,  22],
         [ 55,  33,  22],
         [ 56,  34,  23],
         ...,
         [ 24,  18,  18],
         [ 22,  18,  17],
         [ 22,  18,  17]],

        [[ 55,  33,  22],
         [ 55,  33,  22],
         [ 56,  34,  23],
         ...,
         [ 24,  19,  16],
        

In [42]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [43]:
mean = np.mean(X_train, axis=(0,1,2,3))
std = np.std(X_train, axis=(0,1,2,3))

X_train = (X_train-mean) / (std+1e-7)
X_test = (X_test-mean) / (std+1e-7)

In [45]:
def get_feature_layer(model, data):
	
	total_layers = len(model.layers)
	
	fl_index = total_layers-2
	
	feature_layer_model = keras.Model(
		inputs=model.input,
		outputs=model.get_layer(index=fl_index).output)
	
	feature_layer_output = feature_layer_model.predict(data)
	
	return feature_layer_output

In [ ]:
X_train_feat = get_feature_layer()

In [44]:
model = model(X_train, y_train, X_test, y_test)

ValueError: Please reshape the input data into 2-dimensional matrix.